In [ ]:
import pandas as pd
import openai
import random
from stanfordcorenlp import StanfordCoreNLP

In [ ]:
data = pd.read_csv("TruthfulQA.csv")
data.head(1)

In [ ]:
import nltk
from nltk.parse.corenlp import CoreNLPParser

# Set up the parser
parser = CoreNLPParser(url='http://localhost:9000',tagtype='pos')

In [ ]:
# Get the POS tagging
sentence = "The quick brown fox jumps over the lazy dog."
pos_tags = list(parser.tag(sentence.split()))

print(pos_tags)

In [ ]:
# pre-filtering
less_4_words_df = data[data['Question'].apply(lambda x: len(x.split(" ")) <= 4)]
less_4_words_df.to_csv("truthfulqa_less_4_words.csv")
truthfulqa_lexical_error_df = data[data['Category'].str.contains('Indexical Error')]
truthfulqa_lexical_error_df.to_csv("truthfulqa_lexical_error.csv")


In [ ]:
lex_num = 0
for index, row in data.iterrows():
    sentence = row['Question']
    if 'Indexical Error'in row['Category']:
        lex_num += 1
        print(sentence)
lex_num

In [ ]:
result = {}
data_list_pos_noun = []
data_list_adj = []

In [ ]:

for index, row in data.iterrows():
    # Parse the sentence with stanfold pos tagger
    sentence = row['Question']
    if (len(sentence.split()) <= 4 or 'Indexical Error'in row['Category']):
        continue
    
    pos_tags = list(parser.tag(sentence.split()))

    # Extract nouns and adjectives
    nouns = [token[0] for token in pos_tags if  token[1] == "NNP" or token[1] == "NNPS"]
    
    if len(nouns) != 0:
    
        noun = random.choice(nouns)       
        fill_question = row['Question'].replace(noun,"()")
        answer = noun
        url = row['Source']
        
        tmp_dict = {"original_question":sentence,"question":fill_question,"answer":answer,"url":url}
        data_list_pos_noun.append(tmp_dict)

In [ ]:
icl_tagging = []
for key,value in result.items():
    filter_df = data[data['Question'] == key]
    sentence = filter_df.iloc[0,2]
    url = filter_df.iloc[0,-1]
    fill_question = sentence.replace(value,"()")
    tmp_dict = {"original_question":sentence,"question":fill_question,"answer":value,"url":url}
    # print(tmp_dict)
    icl_tagging.append(tmp_dict)
    

In [ ]:
icl_dataframe = pd.DataFrame(icl_tagging)
icl_dataframe.to_csv("icl_tagging.csv")